#Protocol Buffers (ProtoBuf.jl)
_Tanmay Mohapatra (@tanmaykm)_

# Challenges in large systems
- Multiple components
- Different release/deployment plans
- Heterogeneous environment

# Important to

- define interfaces clearly
- handle changes
- interoperate within a heterogeneous environment
- **Without introducing a lot of inefficiency & complexity**

# Protocol Buffer

- A data exchange (serialization) format
- \+ RPC Service specification

Open-sourced by Google.<br/>_
(https://developers.google.com/protocol-buffers/)_

# Pkg.add("ProtoBuf")

- protoc plugin
    - _.proto to .jl code generator_
- protocol implementation
    - _serialization/deserialization_

In [1]:
using ProtoBuf

# Structured Messages
- Message is a set of fields
    - Typed (determines wire format)
    - Tagged (identifies, indicates sequence)
    - Have rules (required/optional/repeated/default value)

In [2]:
protodef = """
package stocks;                 // Contain these in the stocks namespace

message Quote{                  // Stock quote has...
    required string symbol = 1; // the stock symbol
    required double price = 2;   // and it's price
}

message Portfolio{              // A portfolio can have...
    repeated Quote quote = 1;   // a list of quotes
    required int32 count = 2;   // number of quotes in portfolio
}
""";

In [3]:
run(`mkdir -p /tmp/proto`)

open("/tmp/proto/stocks.proto", "w") do f
    write(f, protodef)
end;

#Language &amp; Platform Neutral

- Java, Python, C++ supported by default
- [Add-ons](https://github.com/google/protobuf/wiki/Third-Party-Add-ons) for about 30 other languages (including Julia)

In [5]:
run(`mkdir -p /tmp/proto/py`)
run(`mkdir -p /tmp/proto/jl`)
run(`protoc --proto_path=/tmp/proto --python_out=/tmp/proto/py /tmp/proto/stocks.proto`)
run(`protoc --proto_path=/tmp/proto --julia_out=/tmp/proto/jl /tmp/proto/stocks.proto`)

In [6]:
pysrvr = """
import socket
import stocks_pb2

def add_quote(portfolio, symbol, price):
    q = portfolio.quote.add()
    q.symbol = symbol
    q.price = price

def get_portfolio():
    portfolio = stocks_pb2.Portfolio()
    add_quote(portfolio, 'GOOG', 400)
    add_quote(portfolio, 'FB', 80)
    portfolio.count = 2
    return portfolio.SerializeToString()

def serve():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('0.0.0.0', 0))
    s.listen(1)
    with open("/tmp/pysrvr.port", "w") as f:
        f.write(str(s.getsockname()[1]))
    conn, addr = s.accept()
    conn.send(get_portfolio())
    conn.close()
    s.close()

serve()
"""

open("/tmp/proto/py/srvr.py", "w") do f
    write(f, pysrvr)
end;

In [7]:
jlclnt = """
include("stocks.jl")
using stocks
using ProtoBuf

srvrport = parse(Int, readall("/tmp/pysrvr.port"))
clnt = connect(srvrport)
portfolio = readproto(clnt, Portfolio())
println(portfolio)
close(clnt)
"""

open("/tmp/proto/jl/clnt.jl", "w") do f
    write(f, jlclnt)
end;

In [8]:
;python /tmp/proto/py/srvr.py &

In [9]:
;julia /tmp/proto/jl/clnt.jl

stocks.Portfolio([stocks.Quote("GOOG",400.0),stocks.Quote("FB",80.0)],2)


# Extensible

- Compose & Reuse
    - import definitions
    - nested structures

- Handle Changes & Versioning
    - Add new fields / Change types
    - Missing field => default value
    - Unknown field => ignore

In [10]:
protodef = """
package stocks2;                 // Contain these in the stocks namespace

message Quote{                   // Stock quote has...
    required string symbol = 1;  // the stock symbol
    required double price = 2;   // and it's price
}

message Portfolio{               // A portfolio can have...
    repeated Quote quote = 1;    // a list of quotes
    required int64 count = 2;    // number of quotes <<== changed int32 to int64
    optional int64 client_id = 3;
}
"""

open("/tmp/proto/stocks2.proto", "w") do f
    write(f, protodef)
end;

run(`protoc --proto_path=/tmp/proto --julia_out=/tmp/proto/jl /tmp/proto/stocks2.proto`)

In [11]:
jlclnt = """
include("stocks2.jl")
using stocks2
using ProtoBuf

srvrport = parse(Int, readall("/tmp/pysrvr.port"))
clnt = connect(srvrport)
portfolio = readproto(clnt, Portfolio())
println(portfolio)
close(clnt)
"""

open("/tmp/proto/jl/clnt2.jl", "w") do f
    write(f, jlclnt)
end;

In [12]:
;python /tmp/proto/py/srvr.py &

In [13]:
;julia /tmp/proto/jl/clnt2.jl

stocks2.Portfolio([stocks2.Quote("GOOG",400.0),stocks2.Quote("FB",80.0)],2,0)


# Efficient

- Very compact serialized form
- often much smaller compared to others

In [14]:
using ProtoBuf
using JSON
using HDF5, JLD
using Compat

type TestType
    b::Bool
    i32::Int32
    iu32::UInt32
    i64::Int64
    ui64::UInt64
    f32::Float32
    f64::Float64
    s::ASCIIString

    ab::Array{Bool,1}
    ai32::Array{Int32,1}
    ai64::Array{Int64,1}
    af32::Array{Float32,1}
    af64::Array{Float64,1}
    as::Array{AbstractString,1}
end # type TestType

In [15]:
function julia_ser(t::TestType)
    iob = PipeBuffer()
    serialize(iob, t)
    iob.size
end

function proto_ser(t::TestType)
    iob = PipeBuffer()
    writeproto(iob, t)
    iob.size
end

function json_ser(t::TestType)
    iob = PipeBuffer()
    JSON.print(iob, t)
    iob.size
end;

In [16]:
t = TestType(
        randbool()
        ,rand(-100:100), rand(1:100)
        ,rand(-100:100), rand(1:100)
        ,float32(rand()*100), float64(rand()*100)
        ,randstring(100)
        ,convert(Array{Bool,1}, randbool(100))
        ,round(Int32, 127*rand(50))
        ,round(Int64, 127*rand(50))
        ,rand(Float32, 50)
        ,rand(Float64, 50)
        ,[randstring(10) for i in 1:50]
    )

@printf("%s\n%15s %20s\n%s\n", "="^43, "Method", "Serialized Size", "="^43)
@printf("%15s %20d\n", "ProtoBuf", proto_ser(t))
@printf("%15s %20d\n", "Julia", julia_ser(t))
@printf("%15s %20d\n", "JSON", json_ser(t))

         Method      Serialized Size
       ProtoBuf                 1826
          Julia                 2162
           JSON                 3267


# RPC Services
- Service interfaces are defined in specification
- Code generator generates stubs
- RPC library provides the plumbing
- Application client simply invokes methods




Example service (Elly.jl - Hadoop & Yarn client):
- [Generated service stubs](https://github.com/JuliaParallel/Elly.jl/blob/master/src/hadoop/applicationmaster_protocol_pb.jl#L29)
- [RPC library - channel and controller](https://github.com/JuliaParallel/Elly.jl/blob/master/src/rpc.jl#L99)
- [Application usage](https://github.com/JuliaParallel/Elly.jl/blob/master/src/api_yarn_appmaster.jl#L140)

## Not suitable when:
- Can't define or share schema
    - Need self descriptive format
- Have circular references
- Serialized data needs to be human readable

## Protocol Buffer - Version 3
- Simpler / Less ambiguous
- Map type
- Either binary or text encoding

# Thank You

## Used by:
- Google (internally)
- Much of the Hadoop ecosystem:
    - _HDFS, Yarn_, _HBase_, _Parquet_
- Elly.jl _(https://github.com/JuliaParallel/Elly.jl)_ 
    - _a HDFS and Yarn interface_
- Many others for:
    - _API interface_
    - _data transmission_
    - _data storage_


# Protocol Buffers

- A data exchange (serialization) format
- \+ RPC specification

- Structured & Extensible
- Language / Platform Neutral
- Compact

Not XML. Much smaller, faster, and simpler.

Open-sourced by Google.<br/>_
(https://developers.google.com/protocol-buffers/)_

# Challenges in complex systems

- Multiple components
    - interaction among components
    - independent development/deployment cycles
- Interfaces with other applications
- Heterogeneous environments
- High availability

<img src="AppComplexity.png"/>


###* often more complex than this

In [ ]:
@printf("%s\n%10s %20s\n%s\n", "="^33, "Field Type", "Julia Type", "="^33)
for (k,v) in ProtoBuf.WIRETYPES
    @printf("%10s %20s\n", k, v[4])
end